In [1]:
import sys, os
repo_start = f'../../'
sys.path.append(repo_start)

from modules.utils.imports import *
from modules.symbolic_net.model_wrapper import model_wrapper
from modules.symbolic_net.build_symbolic_net import symbolic_net
from modules.binn.build_binns_2d_diffusion import BINN
from modules.utils.training_test_split import training_test_split
from modules.analysis.generate_loss_curves import generate_loss_curves
from modules.loaders.format_data import format_data_general
from modules.generate_data.simulate_system import reaction
from modules.symbolic_net.write_terms import write_terms
from modules.symbolic_net.visualize_surface import visualize_surface
from modules.symbolic_net.individual import individual

np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=sys.maxsize)

In [2]:
# Load symbolic net parameters from config file

dir_name = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/equation_fitting/symbolic_net/runs/repeat_10_10k_stop_50_repeats/param_bounds_symbolic_net_l1_reg_0.1_repeat_1_lr_0.001'

config = {}
exec(Path(f'{dir_name}/config.cfg').read_text(encoding="utf8"), {}, config)

training_data_path = config['training_data_path']
species = int(config['species'])
degree = int(config['degree'])
nonzero_term_reg = int(config['nonzero_term_reg'])
l1_reg = float(config['l1_reg'])
param_bounds = float(config['param_bounds'])
density_weight = float(config['density_weight'])
learning_rate = float(config['learning_rate'])

In [3]:
### TRAINING

# Set training hyperparameters
epochs = int(1e6)
rel_save_thresh = 0.05
device = 'cpu'

# Generate training data
training_data = format_data_general(2, 2, file=training_data_path)
uv = training_data[:, -2:]
u_triangle_mesh, v_triangle_mesh = lltriangle(uv[:, 0], uv[:, 1])
u, v = np.ravel(u_triangle_mesh), np.ravel(v_triangle_mesh)

a, b ,k = 1, 1, 0.01
F_true = reaction(u, v, a, b, k)

training_data_nans = np.stack((u, v, F_true), axis=1)

# Remove nans from training data
mask = ~np.isnan(training_data_nans).any(axis=1)
training_data = training_data_nans[mask]

In [4]:
# Load BINN parameters from config file

dir_name = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/binn_testing/debugged_development/comprehensive/runs/2d_repeats/2d_alpha_0.5_repeat_5'
config = {}
exec(Path(f'{dir_name}/config.cfg').read_text(encoding="utf8"), {}, config)

training_data_path = config['training_data_path']
dimensions = int(config['dimensions'])
species = int(config['species'])

density_weight = int(config['density_weight'])

uv_layers = int(config['uv_layers'])
uv_neurons = int(config['uv_neurons'])
f_layers = int(config['f_layers'])
f_neurons = int(config['f_neurons'])

epsilon = float(config['epsilon'])
points = int(config['points'])

diffusion = bool(config['diffusion'])
alpha = float(config['alpha'])

uv_arch = uv_layers * [uv_neurons] + [2]
f_arch = f_layers * [f_neurons] + [1]

In [5]:
# Initialize BINN

binn = BINN(
    species=species, 
    dimensions=dimensions,
    uv_layers=uv_arch, 
    f_layers=f_arch,
    diff=diffusion)

binn.to(device)

opt = torch.optim.Adam(list(binn.parameters()), lr=1e-3)
model = model_wrapper(
    model=binn,
    optimizer=opt,
    loss=binn.loss,
    augmentation=None,
    save_name=f'{dir_name}/binn')

model.load(f"{dir_name}/binn_best_val_model", device=device)

In [6]:
### TRAINING

# Set training hyperparameters
epochs = int(1e6)
rel_save_thresh = 0.05
device = 'cpu'

# Generate training data
training_data = format_data_general(2, 2, file=training_data_path)
uv = training_data[:, -2:]
u_triangle_mesh, v_triangle_mesh = lltriangle(uv[:, 0], uv[:, 1])
u, v = np.ravel(u_triangle_mesh), np.ravel(v_triangle_mesh)

# Calculate true surface to fit to
uv = np.column_stack((np.ravel(u_triangle_mesh), np.ravel(v_triangle_mesh)))
F_true = to_numpy(model.model.reaction(to_torch(uv)[:, None])).flatten()

training_data_nans = np.stack((u, v, F_true), axis=1)

# Remove nans from training data
mask = ~np.isnan(training_data_nans).any(axis=1)
training_data = training_data_nans[mask]

# Split training data
x_train, y_train, x_val, y_val = training_test_split(training_data, 1, device)

In [7]:
# initialize model
device = torch.device('cpu')

binn = BINN(
    species=species, 
    dimensions=dimensions,
    uv_layers=uv_arch, 
    f_layers=f_arch,
    diff=diffusion)

binn.to(device)

opt = torch.optim.Adam(list(binn.parameters()), lr=1e-3)
model = model_wrapper(
    model=binn,
    optimizer=opt,
    loss=binn.loss,
    augmentation=None,
    save_name=f'{dir_name}/binn')

model.load(f"{dir_name}/binn_best_val_model", device=device)


In [8]:
u = x_train[:, 0].flatten()
v = x_train[:, 1].flatten()

hist = torchist.normalize(torchist.histogramdd(x_train, bins=10, 
                            low=[min(u).item(), min(v).item()], 
                            upp=[max(u).item(), max(v).item()]))[0]

edges = torchist.histogramdd_edges(x_train, bins=10, 
                                low=[min(u).item(), min(v).item()], 
                                upp=[max(u).item(), max(v).item()])
edges[0] = edges[0].to(device)
edges[1] = edges[1].to(device)


In [10]:
edges[1]

tensor([0.2023, 0.2845, 0.3666, 0.4488, 0.5310, 0.6132, 0.6953, 0.7775, 0.8597,
        0.9419, 1.0240])

In [16]:
def calc_density(uv, hist, edges):
    uv_indices = torch.zeros((len(uv), len(uv.T)))
    # iterate over columns
    for i in range(len(uv.T)):
        buckets = torch.bucketize(uv[:, i], edges[i], right=True)
        # convert from buckets to indices, correct so max value falls in last bucket
        indices = torch.where(buckets != 0, buckets-1, buckets)
        indices_corrected = torch.where(indices == len(edges[i])-1, indices-1, indices)
        uv_indices[:, i] = indices_corrected
        
    density = torch.tensor([hist[int(indices[0]), int(indices[1])] for indices in uv_indices])
        
    return density

In [18]:
uv = torch.tensor([[5, 0.1], [1, 1], [11, 1]])
calc_density(uv, hist, edges)

tensor([0.0202, 0.0010, 0.0010])

In [5]:
# Load BINN parameters from config file
dir_name = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/binn_testing/debugged_development/comprehensive/runs/2d_repeats/2d_alpha_0.5_repeat_5'
config = {}
exec(Path(f'{dir_name}/config.cfg').read_text(encoding="utf8"), {}, config)

training_data_path = config['training_data_path']
dimensions = int(config['dimensions'])
species = int(config['species'])

density_weight = int(config['density_weight'])

uv_layers = int(config['uv_layers'])
uv_neurons = int(config['uv_neurons'])
f_layers = int(config['f_layers'])
f_neurons = int(config['f_neurons'])

epsilon = float(config['epsilon'])
points = int(config['points'])

diffusion = bool(config['diffusion'])
alpha = float(config['alpha'])

uv_arch = uv_layers * [uv_neurons] + [2]
f_arch = f_layers * [f_neurons] + [1]

# Set training hyperparameters
epochs = int(1e6)
rel_save_thresh = 0.05
device = 'cpu'

# Initialize BINN
binn = BINN(
    species=species, 
    dimensions=dimensions,
    uv_layers=uv_arch, 
    f_layers=f_arch,
    diff=diffusion)

binn.to(device)

opt = torch.optim.Adam(list(binn.parameters()), lr=1e-3)
model = model_wrapper(
    model=binn,
    optimizer=opt,
    loss=binn.loss,
    augmentation=None,
    save_name=f'{dir_name}/binn')

model.load(f"{dir_name}/binn_best_val_model", device=device)

# Generate training data
training_data = format_data_general(2, 2, file=training_data_path)
uv = training_data[:, -2:]
u_triangle_mesh, v_triangle_mesh = lltriangle(uv[:, 0], uv[:, 1])
u, v = np.ravel(u_triangle_mesh), np.ravel(v_triangle_mesh)

# Calculate true surface to fit (F_mlp)
uv = np.column_stack((np.ravel(u_triangle_mesh), np.ravel(v_triangle_mesh)))
F_true = to_numpy(model.model.reaction(to_torch(uv)[:, None])).flatten()

training_data_nans = np.stack((u, v, F_true), axis=1)

# Remove nans from training data
mask = ~np.isnan(training_data_nans).any(axis=1)
training_data = training_data_nans[mask]

# Split training data
x_train, y_train, x_val, y_val = training_test_split(training_data, 1, device)

In [6]:
dir_name = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/equation_fitting/symbolic_net/runs/param_bounds_symbolic_net_l1_reg_0.1_lr_0.001'
config = {}
exec(Path(f'{dir_name}/config.cfg').read_text(encoding="utf8"), {}, config)

training_data_path = config['training_data_path']
species = int(config['species'])
degree = int(config['degree'])
nonzero_term_reg = int(config['nonzero_term_reg'])
l1_reg = float(config['l1_reg'])
param_bounds = float(config['param_bounds'])
density_weight = float(config['density_weight'])
learning_rate = float(config['learning_rate'])

device = 'cpu'
# initialize model and compile
sym_net = symbolic_net(species, degree, param_bounds, device, l1_reg,
                       nonzero_term_reg)
sym_net.to(device)

# initialize optimizer
parameters = sym_net.parameters()
opt = torch.optim.Adam(parameters, lr=learning_rate)

model = model_wrapper(
    model=sym_net,
    optimizer=opt,
    loss=sym_net.loss,
    augmentation=None,
    save_name=f'{dir_name}/binn')